In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install html5lib

In [4]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
import datetime
import requests
from bs4 import BeautifulSoup
from io import StringIO
import time  # ✅ Needed for sleep between batches


In [19]:
from sqlalchemy import create_engine, text

user = 'postgres'
host = 'localhost'
port = '5432'
database = 'smartportfolio'

engine = create_engine(f'postgresql+psycopg2://{user}@{host}:{port}/{database}')

# Use SQLAlchemy's `text()` wrapper
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1")).fetchone()
    print("✅ Connection successful:", result)


✅ Connection successful: (1,)


In [20]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)

# Step 2: Parse the page with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')  # works now after html5lib install
table = soup.find('table', {'id': 'constituents'})

# Step 3: Read the table into a DataFrame
sp500_df = pd.read_html(StringIO(str(table)))[0]

# Step 4: Extract and clean ticker symbols
tickers = sp500_df['Symbol'].tolist()
tickers = [ticker.replace('.', '-') for ticker in tickers]
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DDOG', 

In [21]:
print(f"✅ Loaded {len(tickers)} S&P 500 tickers")

✅ Loaded 503 S&P 500 tickers


In [22]:
end_date= datetime.datetime.today()
start_date= end_date-datetime.timedelta(days=180)

In [23]:
def fetch_and_store(tickers, batch_size=20):
    total = len(tickers)
    for i in range(0, total, batch_size):
        batch = tickers[i:i + batch_size]
        print(f"🔄 Fetching batch {i+1} to {i+len(batch)}")

        try:
            data = yf.download(batch, start=start_date, end=end_date, group_by='ticker', threads=True)
        except Exception as e:
            print(f"❌ Error fetching batch: {e}")
            continue

        for ticker in batch:
            try:
                df = data[ticker].copy()
                df.reset_index(inplace=True)
                df['ticker'] = ticker
                df.columns = [col.lower().replace(' ', '_') for col in df.columns]
                df.rename(columns={'adj_close': 'adj_close'}, inplace=True)

                table_name = f"hist_{ticker.lower().replace('-', '_')}"
                df.to_sql(table_name, engine, if_exists='replace', index=False)

                print(f"✅ Stored: {ticker} → Table: {table_name}")
            except Exception as e:
                print(f"⚠️ Skipped {ticker}: {e}")
        
        time.sleep(3) 


In [24]:
fetch_and_store(tickers, batch_size=20)

🔄 Fetching batch 1 to 20


C:\Users\Riyanshi Kedia\AppData\Local\Temp\ipykernel_24388\2296189172.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(batch, start=start_date, end=end_date, group_by='ticker', threads=True)
[*********************100%***********************]  20 of 20 completed


✅ Stored: MMM → Table: hist_mmm
✅ Stored: AOS → Table: hist_aos
✅ Stored: ABT → Table: hist_abt
✅ Stored: ABBV → Table: hist_abbv
✅ Stored: ACN → Table: hist_acn
✅ Stored: ADBE → Table: hist_adbe
✅ Stored: AMD → Table: hist_amd
✅ Stored: AES → Table: hist_aes
✅ Stored: AFL → Table: hist_afl
✅ Stored: A → Table: hist_a
✅ Stored: APD → Table: hist_apd
✅ Stored: ABNB → Table: hist_abnb
✅ Stored: AKAM → Table: hist_akam
✅ Stored: ALB → Table: hist_alb
✅ Stored: ARE → Table: hist_are
✅ Stored: ALGN → Table: hist_algn
✅ Stored: ALLE → Table: hist_alle
✅ Stored: LNT → Table: hist_lnt
✅ Stored: ALL → Table: hist_all
✅ Stored: GOOGL → Table: hist_googl


C:\Users\Riyanshi Kedia\AppData\Local\Temp\ipykernel_24388\2296189172.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(batch, start=start_date, end=end_date, group_by='ticker', threads=True)
[************          25%                       ]  5 of 20 completed

🔄 Fetching batch 21 to 40


...

[*********************100%***********************]  20 of 20 completed


✅ Stored: WBA → Table: hist_wba
✅ Stored: WMT → Table: hist_wmt
✅ Stored: DIS → Table: hist_dis
✅ Stored: WBD → Table: hist_wbd
✅ Stored: WM → Table: hist_wm
✅ Stored: WAT → Table: hist_wat
✅ Stored: WEC → Table: hist_wec
✅ Stored: WFC → Table: hist_wfc
✅ Stored: WELL → Table: hist_well
✅ Stored: WST → Table: hist_wst
✅ Stored: WDC → Table: hist_wdc
✅ Stored: WY → Table: hist_wy
✅ Stored: WSM → Table: hist_wsm
✅ Stored: WMB → Table: hist_wmb
✅ Stored: WTW → Table: hist_wtw
✅ Stored: WDAY → Table: hist_wday
✅ Stored: WYNN → Table: hist_wynn
✅ Stored: XEL → Table: hist_xel
✅ Stored: XYL → Table: hist_xyl
✅ Stored: YUM → Table: hist_yum


C:\Users\Riyanshi Kedia\AppData\Local\Temp\ipykernel_24388\2296189172.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(batch, start=start_date, end=end_date, group_by='ticker', threads=True)
[*********************100%***********************]  3 of 3 completed

🔄 Fetching batch 501 to 503


✅ Stored: ZBRA → Table: hist_zbra
✅ Stored: ZBH → Table: hist_zbh
✅ Stored: ZTS → Table: hist_zts
